In [1]:
import h5py
import librosa
import numpy as np

file = h5py.File('E:/dataset_6ms.h5', 'r')

print(file['label'].shape)
# labels = file['label'][:36406].squeeze()

gender_labels = file['gender'][:3000].squeeze().astype(str)
print('done!')
male_indexes =  np.where(gender_labels == 'male')
femmale_indexes =  np.where(gender_labels == 'female')

print('done!')
y = file['label'][male_indexes]
print('done!')
X = file['stft'][male_indexes]
print('done!')

X.shape, y.shape

(44344, 1)
done!
done!
done!
done!


((1482, 2049, 50), (1482, 1))

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((992, 2049, 50), (490, 2049, 50), (992, 1), (490, 1))

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, LeakyReLU, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras.layers import Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, Reshape, AveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import adam_v2
import keras

# import rms prop from keras


es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
opt = adam_v2.Adam(learning_rate=0.0001)
# model.compile(loss='categorical_crossentropy', optimizer=opt)

feature_shape = X_train[0].shape
print(feature_shape)
model = Sequential()
model.add(Reshape(target_shape=(
    feature_shape[0], feature_shape[1], 1), input_shape=(feature_shape[0], feature_shape[1])))
# 1st conv layer
model.add(Conv2D(16, (5, 5), strides=(2, 1)))
model.add(BatchNormalization())
model.add(LeakyReLU())
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(0.2))


# 2nd conv layer
model.add(Conv2D(32, (3, 3), strides=(2, 1)))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 1), padding='same'))
model.add(Dropout(0.2))

# 3rd conv layer
model.add(Conv2D(64, (3, 1), strides=(2, 1)))
model.add(LeakyReLU())
model.add(BatchNormalization())
model.add(Dropout(0.2))

# 4th conv layer
model.add(Conv2D(128, (3, 3), strides=(2, 1), padding='same'))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# flatten the output and add a dense layer
# model.add(GlobalMaxPooling2D())
model.add(Flatten())
model.add(Dropout(0.2))
# model.add(Dense(1024, activation='relu'))
model.add(Dense(512))
model.add(LeakyReLU())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
# global pooling
# outout layer
model.add(Dense(6, activation='sigmoid'))
model.add(Dense(6, activation='softmax'))
model.summary()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt, metrics=['accuracy'])


(2049, 50)
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_3 (Reshape)         (None, 2049, 50, 1)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 1023, 46, 16)      416       
                                                                 
 batch_normalization_12 (Bat  (None, 1023, 46, 16)     64        
 chNormalization)                                                
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 1023, 46, 16)      0         
                                                                 
 average_pooling2d_1 (Averag  (None, 512, 23, 16)      0         
 ePooling2D)                                                     
                                                                 
 dropout_18 (Dropout)        (None, 512, 23

In [9]:
cnnhistory=model.fit(X_train, y_train, batch_size=200, epochs=200,validation_data=(X_test, y_test), callbacks=[es], verbose=1)


Epoch 1/200
5/5 [==============================] - 4s 580ms/step - loss: 1.8154 - accuracy: 0.1784 - val_loss: 1.8134 - val_accuracy: 0.1980
Epoch 2/200
5/5 [==============================] - 2s 351ms/step - loss: 1.7485 - accuracy: 0.2379 - val_loss: 1.8082 - val_accuracy: 0.1980
Epoch 3/200
5/5 [==============================] - 2s 350ms/step - loss: 1.7043 - accuracy: 0.2762 - val_loss: 1.7985 - val_accuracy: 0.1959
Epoch 4/200
5/5 [==============================] - 2s 352ms/step - loss: 1.6704 - accuracy: 0.3155 - val_loss: 1.8132 - val_accuracy: 0.1980
Epoch 5/200
5/5 [==============================] - 2s 348ms/step - loss: 1.6471 - accuracy: 0.3044 - val_loss: 1.8084 - val_accuracy: 0.1898
Epoch 6/200
5/5 [==============================] - 2s 348ms/step - loss: 1.6224 - accuracy: 0.3276 - val_loss: 1.7935 - val_accuracy: 0.2184
Epoch 7/200
5/5 [==============================] - 2s 358ms/step - loss: 1.5963 - accuracy: 0.3508 - val_loss: 1.8025 - val_accuracy: 0.1898
Epoch 8/200
5

In [ ]:
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(cnnhistory.history['accuracy'])
plt.plot(cnnhistory.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
cnnhistory.model.save('cnn_60.h5')